# Predicting Subscription of Game-Related Newsletter using Classification

## (1) Introduction

The data was collected by a research group at UBC in Computer Science, led by Frank Wood (Pacific Laboratory for Artificial Intelligence (PLAI), 2023). They set up a MineCraft server that links to an external site and recorded the players' actions as they navigated through the world.  

The datasets are not in a tidy format. The "end_time" and "start_time" has to be separated into two columns in sessions.csv: date and time. There are missing values in the age column of players.csv, so they should be removed. 

### Question 
Can the player age and hours played in MineCraft predict whether players are subscribed to a game-related newsletter? 

### Players Dataset
Overview
- Number of observations (rows): 196
- Number of variables (columns): 7
- played_hours might be from the Minecraft server, but all other data was most likely from their website where players enter their information

| Variable Name | Type | Description | Summary Statistics (to 2 decimals) | Notes / Issues |
| - | - | - | - | - |
| `experience` | Categorical | Player’s self-reported experience level (e.g., Amateur, Intermediate, Expert, etc.) | 5 unique values; most common: *Amateur* (63 players) | Could be ordinal but stored as text. |
| `subscribe` | Categorical | Whether the player subscribed to a game-related newsletter | 144 (True), 52 (False) | Fairly imbalanced — may bias models. |
| `hashedEmail` | String | Unique player identifier (hashed for privacy) | 196 unique values | Good for joining with `sessions.csv`. |
| `played_hours` | Numeric | Total number of hours each player spent in-game | Mean = 5.85, SD = 28.36, Min = 0.00, Median = 0.10, Max = 223.10 | Highly skewed — some extreme outliers. |
| `name` | String | Player’s name (pseudonymized) | 196 unique values | Not useful analytically. |
| `gender` | Categorical | Self-reported gender (Male, Female, Nonbinary, etc.) | 7 unique categories; most common: *Male* (124 players) | Possible inconsistencies or typos in text. |
| `Age` | Numeric | Player’s age in years | Mean = 21.14, SD = 7.39, Min = 9.00, Median = 19.00, Max = 58.00 | 2 missing values; wide age range. |

### Sessions Dataset
Overview
- Number of observations (rows): 1,535
- Number of variables (columns): 5
- Data was collected from the Minecraft server.

| Variable Name | Type | Description | Summary Statistics (to 2 decimals) | Notes / Issues |
| - | - | - | - | - |
| `hashedEmail` | String | Unique identifier matching players in `players.csv` | 125 unique IDs; most common appears 310 times | Used to link sessions to players. |
| `start_time` | String | Start timestamp of a game session (formatted date-time string) | 1,504 unique; most common = “28/06/2024 01:31” | Needs conversion to datetime. |
| `end_time` | String | End timestamp of a game session | 1,489 unique; 2 missing values | Needs conversion to datetime; missing values cause incomplete sessions. |
| `original_start_time` | Numeric | Original Unix timestamp of session start | Mean = 1.72×10¹², SD = 3.56×10⁹ | Likely milliseconds since epoch; can be converted to readable dates. |
| `original_end_time` | Numeric | Original Unix timestamp of session end | Mean = 1.72×10¹², SD = 3.55×10⁹ | 2 missing values; aligns with `end_time`. |


## (2) Methods and Results

### Method

1. Load the players.csv dataset then tidy it by removing missing values in the age column.
2. Create a new tibble called “subscribe_data“ with variables “age”, “played_hours”, and “subscribe”.
3. Find mean, max, and min of “played_hours” and “age”.
4. Create a scatterplot of the “subscribe_data” to visualize.
5. Split the dataset into testing （30%) & training (70%) datasets
6. Conduct cross-validation on the training set
7. Split training data into subtrain and validation set
8. Preprocess data: scale and center subtraining data and create a recipe.
9. Train classifier: set specification model with tune() instead of a K value, create the workflow, and fit the training data
10. Predict labels in the validation set.
11. Evaluate the performance (accuracy), then make a plot (Accuracy vs. K) to choose the best K value.  

In [5]:
# include libraries
library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)
library(ggplot2)
options(repr.matrix.max.rows = 6)

In [6]:
# load both datasets
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")
players
sessions

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
⋮,⋮,⋮,⋮,⋮
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,28/07/2024 15:36,28/07/2024 15:57,1.72218e+12,1.72218e+12
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,25/07/2024 06:15,25/07/2024 06:22,1.72189e+12,1.72189e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,20/05/2024 02:26,20/05/2024 02:45,1.71617e+12,1.71617e+12


In [7]:
# select variables that will be used and filter out rows that have missing values
players_tidy <- players |>
  select(subscribe, played_hours, Age) |>
  filter(!is.na(Age))

### Summary of Variables
| Variable Name | Mean | Max | Min|
| - | - | - | - |
| played_hours | 5.85 | 223.10 | 0.00 |
| Age | 21.14 | 58.00 | 9.00 |

## (3) Discussion

## (4) References

Pacific Laboratory for Artificial Intelligence (PLAI). (2023, September 28). Home Page - Pacific Laboratory for Artificial Intelligence. Pacific Laboratory for Artificial Intelligence. https://plai.cs.ubc.ca/